In [14]:
import numpy as np
from momaapi import MOMA
from itertools import product
from tqdm import tqdm

In [2]:
moma = MOMA(dir_moma="/data/dir_moma", paradigm="standard")

In [6]:
act2seq = {} # activity name -> sub-activity sequence list
sact2id = {} # sub-activity name -> sub-activity id

for split in ["train", "val", "test"]:
    ids_act = moma.get_ids_act(split=split)
    for act in tqdm(moma.get_anns_act(ids_act=ids_act), desc=f"[{split}]"):
        seq = []
        for sact in moma.get_anns_sact(ids_sact=act.ids_sact):
            seq.append(sact.cid)
            sact2id[sact.cname] = sact.cid
        
        if act.cname not in act2seq:
            act2seq[act.cname] = [seq]
        else:
            act2seq[act.cname].append(seq)

sact_to_act = moma.taxonomy["sact_to_act"]
act_to_sact = {}
for sact, act in sact_to_act.items():
    if act not in act_to_sact:
        act_to_sact[act] = [sact2id[sact]]
    else:
        act_to_sact[act].append(sact2id[sact])

[test]: 100%|██████████| 282/282 [00:09<00:00, 30.55it/s] 


In [52]:
act_cname = "basketball game"
seq_list = act2seq[act_cname]

k = 2
sact_set = act_to_sact[act_cname]
motifs = list(product(sact_set, repeat=k))
motifs = [list(m) for m in motifs]
count = np.zeros(len(motifs))

for i, motif in enumerate(tqdm(motifs)):
    m = len(motif)
    for seq in seq_list:
        seq_len = len(seq)
        if seq_len < m:
            continue
        for j in range(0, seq_len - m + 1):
            if seq[j:j+m] == motif:
                count[i] += 1

prob = count / count.sum()
sorted_idx = np.argsort(prob)[::-1]

for idx in sorted_idx:
    print(f"motif: {motifs[idx]} prob: {prob[idx] * 100.:.3f}% ({int(count[idx])})") 

100%|██████████| 16/16 [00:00<00:00, 669.17it/s]

motif: [62, 62] prob: 29.087% (774)
motif: [22, 22] prob: 19.241% (512)
motif: [62, 22] prob: 10.560% (281)
motif: [22, 62] prob: 9.508% (253)
motif: [62, 24] prob: 6.764% (180)
motif: [22, 24] prob: 5.862% (156)
motif: [24, 62] prob: 5.111% (136)
motif: [24, 24] prob: 5.111% (136)
motif: [24, 22] prob: 5.111% (136)
motif: [23, 23] prob: 0.714% (19)
motif: [24, 23] prob: 0.639% (17)
motif: [23, 24] prob: 0.601% (16)
motif: [22, 23] prob: 0.526% (14)
motif: [23, 62] prob: 0.451% (12)
motif: [62, 23] prob: 0.376% (10)
motif: [23, 22] prob: 0.338% (9)


In [29]:
x = np.array([1, 4, 2, 3, 7, 5])
y = ["A", "B", "C", "D", "E", "F"]

idx = np.argsort(x)[::-1]

sx = [x[i] for i in idx]
sy = [y[i] for i in idx]

print(sx)
print(sy)

[7, 5, 4, 3, 2, 1]
['E', 'F', 'B', 'D', 'C', 'A']
